<a href="https://colab.research.google.com/github/joeyyy688/flower_predictor/blob/master/cnn_flowerpredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import os


In [6]:
_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz';

zip_file = tf.keras.utils.get_file(origin = _url, fname = 'flower_photos.tgz', extract = True, cache_subdir = '/content');

basedir = os.path.join(os.path.dirname(zip_file), 'flower_photos');

228827136/228813984 [==============================] - 3s 0us/step


In [7]:
BATCH_SIZE = 64
IMAGE_SIZE = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

train_generator = datagen.flow_from_directory(
    basedir, target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
     subset = 'training'
)

val_generator = datagen.flow_from_directory(
    basedir, target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)



Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [8]:
print(train_generator.class_indices)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [9]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f : 
  f.write(labels);

In [10]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top = False, weights='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [11]:
base_model.trainable = False

In [12]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation = 'relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
epochs = 10

history = model.fit(train_generator, epochs=epochs, validation_data= val_generator)

Epoch 1/10
46/46 [==============================] - 52s 394ms/step - loss: 0.6557 - accuracy: 0.7669 - val_loss: 0.4672 - val_accuracy: 0.8372
Epoch 2/10
46/46 [==============================] - 16s 338ms/step - loss: 0.2623 - accuracy: 0.9075 - val_loss: 0.4388 - val_accuracy: 0.8454
Epoch 3/10
46/46 [==============================] - 16s 338ms/step - loss: 0.1616 - accuracy: 0.9452 - val_loss: 0.4122 - val_accuracy: 0.8714
Epoch 4/10
46/46 [==============================] - 15s 335ms/step - loss: 0.0945 - accuracy: 0.9707 - val_loss: 0.4316 - val_accuracy: 0.8509
Epoch 5/10
46/46 [==============================] - 15s 335ms/step - loss: 0.0731 - accuracy: 0.9813 - val_loss: 0.4557 - val_accuracy: 0.8700
Epoch 6/10
46/46 [==============================] - 15s 334ms/step - loss: 0.0492 - accuracy: 0.9874 - val_loss: 0.4885 - val_accuracy: 0.8646
Epoch 7/10
46/46 [==============================] - 16s 337ms/step - loss: 0.0430 - accuracy: 0.9878 - val_loss: 0.5548 - val_accuracy: 0.8523

In [15]:
import tensorflow as tf
import keras

saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

convertor = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

tflite_model = convertor.convert()

with open('model.tflite', 'wb') as f :
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


In [16]:
from google.colab import files

files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>